# Predicting arrival delay using Linear Regression

In [ ]:
# !pip install pyspark

In [102]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

In [103]:
# importing necessary packages
import pandas as pd
#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression

In [104]:
# creating spark session
spark = SparkSession.builder.appName("delay").getOrCreate()

In [105]:
pandas_df = pd.read_csv("Allmonths_Flight_Data.csv", header=0)
pandas_df = pandas_df.drop([
    'ACTUAL_ELAPSED_TIME',
 'AIR_TIME',
 'ARR_DEL15',
 'ARR_DELAY',
 'ARR_TIME',
 'ARR_TIME_BLK',
## 'CANCELLATION_CODE',
 'CANCELLED',
## 'CARRIER_DELAY',
#  'CRS_ARR_TIME',
#  'CRS_DEP_TIME',
#  'CRS_ELAPSED_TIME',
#  'DAY_OF_WEEK',
 'DEP_DEL15',
#  'DEP_DELAY',
 'DEP_TIME',
 'DEP_TIME_BLK',
#  'DEST',
#  'DISTANCE',
## 'DIVERTED',
 'FL_DATE',
## 'LATE_AIRCRAFT_DELAY',
#  'MONTH',
## 'NAS_DELAY',
#  'OP_CARRIER_FL_NUM',
#  'OP_UNIQUE_CARRIER',
#  'ORIGIN',
## 'SECURITY_DELAY',
 'TAIL_NUM',
 'TAXI_IN',
 'TAXI_OUT',
## 'WEATHER_DELAY',
 'WHEELS_OFF',
 'WHEELS_ON',
#  'YEAR'
],axis=1)

In [106]:
def classify(num):
    if (num < 0):
        return ('No Delay') 
    else:
        if (num < 15):
            return ('Late: < 15 mins')
        elif (num < 30):
            return ('Late: < 30 mins')
        elif (num < 60):
            return ('Late: < 1 hours')
        elif (num < 90):
            return ('Late: < 1.5 hours')
        elif (num < 120):
            return ('Late: < 2 hours')
        elif (num < 150):
            return ('Late: < 2.5 hours')
        elif (num < 180):
            return ('Late: < 3 hours')
        elif (num < 240):
            return ('Late: < 4 hours')
        elif (num < 300):
            return ('Late: < 5 hours')
        else:
            return ('Late: > 5 hours')


# Transforming arrival delay to fall into one of 5 categories
pandas_df['DEP_DELAY'] = pandas_df['DEP_DELAY'].apply(lambda x: classify(x))

In [107]:
spark_df = spark.createDataFrame(pandas_df)

/opt/anaconda3/envs/flights_venv/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/opt/anaconda3/envs/flights_venv/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [108]:
pandas_df.columns

Index(['YEAR', 'MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER',
       'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'DEP_DELAY',
       'CRS_ARR_TIME', 'CRS_ELAPSED_TIME', 'DISTANCE'],
      dtype='object')

In [109]:
# from pyspark.ml.feature import OneHotEncoder
# ohe = OneHotEncoder(inputCol="OP_UNIQUE_CARRIER", outputCol="OP_UNIQUE_CARRIER_numeric")
# spark_df = ohe.fit(spark_df).transform(spark_df)

In [110]:
from pyspark.ml.feature import StringIndexer
indexer1 = StringIndexer(inputCol="OP_UNIQUE_CARRIER", outputCol="OP_UNIQUE_CARRIER_numeric")
#Fits a model to the input dataset with optional parameters.
spark_df = indexer1.fit(spark_df).transform(spark_df)

indexer2 = StringIndexer(inputCol="ORIGIN", outputCol="ORIGIN_numeric")
spark_df = indexer2.fit(spark_df).transform(spark_df)

indexer3 = StringIndexer(inputCol="DEST", outputCol="DEST_numeric")
spark_df = indexer3.fit(spark_df).transform(spark_df)

indexer4 = StringIndexer(inputCol="DEP_DELAY", outputCol="DEP_DELAY_numeric")
spark_df = indexer4.fit(spark_df).transform(spark_df)

spark_df.show()

22/10/09 12:29:12 WARN TaskSetManager: Stage 75 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/09 12:29:13 WARN TaskSetManager: Stage 78 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/09 12:29:13 WARN TaskSetManager: Stage 81 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/09 12:29:14 WARN TaskSetManager: Stage 84 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/09 12:29:14 WARN TaskSetManager: Stage 87 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.


22/10/09 12:29:18 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 87 (TID 145): Attempting to kill Python Worker
+----+-----+-----------+-----------------+-----------------+------+----+------------+-----------------+------------+----------------+--------+-------------------------+--------------+------------+-----------------+
|YEAR|MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|        DEP_DELAY|CRS_ARR_TIME|CRS_ELAPSED_TIME|DISTANCE|OP_UNIQUE_CARRIER_numeric|ORIGIN_numeric|DEST_numeric|DEP_DELAY_numeric|
+----+-----+-----------+-----------------+-----------------+------+----+------------+-----------------+------------+----------------+--------+-------------------------+--------------+------------+-----------------+
|2022|    1|          3|               YX|             4904|   LGA| BNA|         620|         No Delay|         807|           167.0|   764.0|                      0.0|           0.0|        21.0|              0.0|
|20


[Stage 87:==========================================================(1 + 0) / 1]



In [111]:
assembler = VectorAssembler(inputCols = ['YEAR', 'MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER_numeric',
       'OP_CARRIER_FL_NUM', 'ORIGIN_numeric', 'DEST_numeric', 'CRS_DEP_TIME',
       'CRS_ARR_TIME', 'CRS_ELAPSED_TIME', 'DISTANCE'], outputCol='features')
output = assembler.transform(spark_df)

In [112]:
finalised_data = output.select('features', 'DEP_DELAY_numeric')
finalised_data.show()

22/10/09 12:29:19 WARN TaskSetManager: Stage 88 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.


22/10/09 12:29:23 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 88 (TID 146): Attempting to kill Python Worker
+--------------------+-----------------+
|            features|DEP_DELAY_numeric|
+--------------------+-----------------+
|[2022.0,1.0,3.0,0...|              0.0|
|[2022.0,1.0,4.0,0...|              0.0|
|[2022.0,1.0,5.0,0...|              2.0|
|[2022.0,1.0,7.0,0...|              0.0|
|[2022.0,1.0,1.0,0...|              1.0|
|[2022.0,1.0,2.0,0...|              0.0|
|[2022.0,1.0,3.0,0...|              0.0|
|[2022.0,1.0,4.0,0...|              4.0|
|[2022.0,1.0,5.0,0...|              5.0|
|[2022.0,1.0,7.0,0...|              0.0|
|[2022.0,1.0,1.0,0...|              0.0|
|[2022.0,1.0,2.0,0...|              3.0|
|[2022.0,1.0,3.0,0...|              0.0|
|[2022.0,1.0,4.0,0...|              0.0|
|[2022.0,1.0,5.0,0...|              0.0|
|[2022.0,1.0,1.0,0...|              0.0|
|[2022.0,1.0,6.0,0...|              2.0|
|[2022.0,1.0,7.0,0...|              1.0|
|[

In [113]:
train, test = finalised_data.randomSplit([0.8, 0.2])

In [114]:
lr = LogisticRegression(labelCol='DEP_DELAY_numeric',featuresCol='features')
lrn = lr.fit(train)

22/10/09 12:29:23 WARN TaskSetManager: Stage 89 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.


22/10/09 12:29:26 WARN TaskSetManager: Stage 90 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.


22/10/09 12:29:28 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/10/09 12:29:28 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


22/10/09 12:29:28 WARN TaskSetManager: Stage 91 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/09 12:29:28 WARN TaskSetManager: Stage 92 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/09 12:29:28 WARN TaskSetManager: Stage 93 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/09 12:29:28 WARN TaskSetManager: Stage 94 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/09 12:29:28 WARN TaskSetManager: Stage 95 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/09 12:29:28 WARN TaskSetManager: Stage 96 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/09 12:29:29 WARN TaskSetManager: Stage 97 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/

22/10/09 12:29:35 WARN TaskSetManager: Stage 149 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/09 12:29:35 WARN TaskSetManager: Stage 150 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/09 12:29:35 WARN TaskSetManager: Stage 151 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/09 12:29:35 WARN TaskSetManager: Stage 152 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/09 12:29:35 WARN TaskSetManager: Stage 153 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/09 12:29:35 WARN TaskSetManager: Stage 154 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
22/10/09 12:29:35 WARN TaskSetManager: Stage 155 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.

In [115]:
lrn_summary = lrn.summary
lrn_summary.predictions.show()

22/10/09 12:30:14 WARN TaskSetManager: Stage 194 contains a task of very large size (3337 KiB). The maximum recommended task size is 1000 KiB.
+--------------------+-----------------+--------------------+--------------------+----------+
|            features|DEP_DELAY_numeric|       rawPrediction|         probability|prediction|
+--------------------+-----------------+--------------------+--------------------+----------+
|[2022.0,1.0,1.0,0...|              0.0|[3.19183601791997...|[0.63223992568620...|       0.0|
|[2022.0,1.0,1.0,0...|              0.0|[3.19183601791997...|[0.63223992568620...|       0.0|
|[2022.0,1.0,1.0,0...|              3.0|[3.19183601791997...|[0.63223992568620...|       0.0|
|[2022.0,1.0,1.0,0...|              5.0|[3.18838187998621...|[0.63144522217886...|       0.0|
|[2022.0,1.0,1.0,0...|              0.0|[4.10276795675082...|[0.79845986166389...|       0.0|
|[2022.0,1.0,1.0,0...|              0.0|[4.10276795675082...|[0.79845986166389...|       0.0|
|[2022.0,1.